In [239]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Segmentation and grouping of neighborhoods in the city of Toronto

## Import Toronto information into Wikipedia

In [323]:
toro=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=toro[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [324]:
#we search and identify all the rows that we must discard
n=[]
for x in range(180):
    num=df.iloc[x][2]
    if num=="Not assigned":
        n.append(x)

In [325]:
for i in range (77):
    df=df.drop(n[i])

df.iloc[56][2]='Mount Dennis'
df.iloc[76][2]='Mississauga'
df.iloc[92][2]='Downtown '
df.iloc[100][2]='East Toronto'


In [326]:
#DataFrame is Clean
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [327]:
# The addresses are built to be able to find the longitude and latitude of each one of them
address=[]
for x in range (103):
    a=df.iloc[x][2]
    b = a.split(", ")
    h = b[0].split("-")
    c=h[0] + ", " + "Toronto, Canada"
    address.append(c)


In [328]:
#We look for the longitudes and latitudes of the directions
lat=[]
lon=[]

for x in range (103):
    geolocator = Nominatim(user_agent="t_explorer")
    location = geolocator.geocode(address[x])
    latitude = location.latitude
    longitude = location.longitude
    lat.append(latitude)
    lon.append(longitude)

In [329]:
#Añadimos la informacion al dataframe
df['latitude'] = lat
df['longitude'] = lon
df.head()

,Postal Code,Borough,Neighbourhood,latitude,longitude
2,M3A,North York,Parkwoods,43.758800,-79.320197
3,M4A,North York,Victoria Village,43.732658,-79.311189
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.659659,-79.390340


In [330]:
#Buscamos la ubicacion de la ciudad de Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [331]:
#Create a map of Toronto with neighborhoods superimposed on top.

map_t=folium.Map(location=[latitude, longitude], zoom_start=10)
x=0

# add markers to map

for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t)
    
map_t

In [332]:
#identificamos las diferentes partes de Toronto
df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [349]:
# creamos un DF de Downtown Toronto
ny_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
n=ny_data['Neighbourhood']
ny_data=ny_data.drop(['Neighbourhood'], axis = 'columns')
ny_data['Neighborhood']=n
ny_data.head()

,Postal Code,Borough,latitude,longitude,Neighborhood
0,M5A,Downtown Toronto,43.660706,-79.360457,"Regent Park, Harbourfront"
1,M7A,Downtown Toronto,43.659659,-79.390340,"Queen's Park, Ontario Provincial Government"
2,M5B,Downtown Toronto,43.656500,-79.377114,"Garden District, Ryerson"
3,M5C,Downtown Toronto,43.669403,-79.372704,St. James Town
4,M5E,Downtown Toronto,43.647984,-79.375396,Berczy Park


In [350]:
#Buscams las coordenadas de North York
address = 'Downtown Toronto, Toronto. Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [351]:
# create map of Downtown Toronto using latitude and longitude values
map_ny = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(ny_data['latitude'], ny_data['longitude'], ny_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

In [352]:
CLIENT_ID = 'LRGCXLZ3GZCILAWSLKL23EMPKIKDKNG5HOXS10QQUO2MHB2J' # your Foursquare ID
CLIENT_SECRET = '5DEKXD4LLKJYT1UH2GNAVMMEZZ0HX2QYQCD4A2P5UMHOFG53' # your Foursquare Secret
VERSION = '20190425' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LRGCXLZ3GZCILAWSLKL23EMPKIKDKNG5HOXS10QQUO2MHB2J
CLIENT_SECRET:5DEKXD4LLKJYT1UH2GNAVMMEZZ0HX2QYQCD4A2P5UMHOFG53


In [353]:
neighborhood_latitude = ny_data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = ny_data.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = ny_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6607056, -79.3604569.


In [354]:
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display UR

'https://api.foursquare.com/v2/venues/explore?&client_id=LRGCXLZ3GZCILAWSLKL23EMPKIKDKNG5HOXS10QQUO2MHB2J&client_secret=5DEKXD4LLKJYT1UH2GNAVMMEZZ0HX2QYQCD4A2P5UMHOFG53&v=20190425&ll=43.6607056,-79.3604569&radius=500&limit=100'

In [355]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6029a8c2dba4b9223dbd9ea1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Regent Park',
  'headerFullLocation': 'Regent Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 43.6652056045, 'lng': -79.35424823370592},
   'sw': {'lat': 43.6562055955, 'lng': -79.36666556629409}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5092fe87e4b0476c6e7375e2',
       'name': 'Regent Park Aquatic Centre',
       'location': {'address': '640 Dundas St. East',
        'crossStreet': 'at Sumach St.',
        'lat': 43.6606003130796,
        'lng': -79.36139162915349,
        'labeledLatLngs'

In [356]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [357]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-357-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Sumach Espresso,Coffee Shop,43.658135,-79.359515
2,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808
3,Paintbox Bistro,Restaurant,43.660050,-79.362855
4,Thai To Go,Thai Restaurant,43.663418,-79.360710


In [358]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown

In [360]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [361]:
downt_venues = getNearbyVenues(names=ny_data['Neighborhood'],
                                   latitudes=ny_data['latitude'],
                                   longitudes=ny_data['longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Downtown 
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [362]:
print(downt_venues.shape)
downt_venues.head()

(1277, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,"Regent Park, Harbourfront",43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
2,"Regent Park, Harbourfront",43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
3,"Regent Park, Harbourfront",43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant
4,"Regent Park, Harbourfront",43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant


In [364]:
downt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",56,56,56,56,56,56
Central Bay Street,100,100,100,100,100,100
Christie,59,59,59,59,59,59
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown,92,92,92,92,92,92
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",62,62,62,62,62,62


In [365]:
print('There are {} uniques categories.'.format(len(downt_venues['Venue Category'].unique())))

There are 200 uniques categories.


# Analyze Each Neighborhood

In [366]:
# one hot encoding
dt_onehot = pd.get_dummies(downt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = downt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Yoga Studio,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0

In [367]:
dt_onehot.shape

(1277, 200)

In [368]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,History Museum,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.010000,0.010000,0.0,0.01,0.000000,0.02,0.0,0.0,0.00,0.0,0.010000,0.01,0.03,0.0,0.000000,0.000000,0.0,0.0,0.030000,0.000000,0.0,0.0,0.010000,0.000000,0.00,0.030000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.02,0.000000,0.01,0.000000,0.040000,0.080000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.01,0.00,0.010000,0.000000,0.010000,0.020000,0.0,0.000000,0.010000,0.01,0.000000,0.010000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.010000,0.00,0.000000,0.0,0.02,0.000000,0.0,0.01,0.000000,0.0,0.01,0.01,0.01,0.0,0.00,0.0,0.0,0.000000,0.030000,0.00,0.0,0.0,0.0,0.000000,0.0,0.01,0.000000,0.010000,0.030000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.040000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.040000,0.040000,0.01,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.01,0.000000,0.010000,0.01,0.0,0.01,0.00,0.0,0.00,0.0,0.01,0.0,0.0,0.000000,0.020000,0.0,0.0,0.0,0.01,0.010000,0.0,0.000000,0.01,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.04,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.01,0.0,0.000000,0.000000,0.000000,0.0000

In [369]:
dt_grouped.shape

(19, 200)

In [370]:
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1          Café  0.05
2    Restaurant  0.05
3  Cocktail Bar  0.04
4         Hotel  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
            venue  freq
0           Hotel  0.11
1     Coffee Shop  0.07
2     Pizza Place  0.05
3             Gym  0.05
4  Scenic Lookout  0.05


----Central Bay Street----
                venue  freq
0  Italian Restaurant  0.05
1         Coffee Shop  0.05
2            Boutique  0.05
3               Hotel  0.04
4   French Restaurant  0.04


----Christie----
                venue  freq
0   Korean Restaurant  0.24
1         Coffee Shop  0.05
2  Mexican Restaurant  0.03
3       Grocery Store  0.03
4                Café  0.03


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.07
2     Sushi Restaurant  0.07
3           Restaurant  0.04
4              Gay Bar  0.0

In [371]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [372]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Breakfast Spot,Bakery
1,"CN Tower, King and Spadina, Railway Lands, Har...",Hotel,Coffee Shop,Gym,Baseball Stadium,Bar,Pizza Place,Scenic Lookout,Ice Cream Shop,Aquarium,Yoga Studio
2,Central Bay Street,Boutique,Coffee Shop,Italian Restaurant,Café,Restaurant,French Restaurant,Spa,Hotel,Sushi Restaurant,Clothing Store
3,Christie,Korean Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Ice Cream Shop,Café,Cocktail Bar,Karaoke Bar,Grocery Store,Dessert Shop
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Burger Joint,Fast Food Restaurant,Café,Hotel,Yoga Studio


# Cluster Neighborhoods

In [397]:
# set number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 1])

In [398]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_merged = ny_data

# merge dt_grouped with dt_data to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dt_merged.head() # check the last columns!

,Postal Code,Borough,latitude,longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,43.660706,-79.360457,"Regent Park, Harbourfront",1,Coffee Shop,Restaurant,Thai Restaurant,Pet Store,Pharmacy,Beer Store,Pool,Food Truck,Electronics Store,Sushi Restaurant
1,M7A,Downtown Toronto,43.659659,-79.390340,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,French Restaurant,Bank,Vegetarian / Vegan Restaurant,Bubble Tea Shop,Thai Restaurant,Japanese Restaurant
2,M5B,Downtown Toronto,43.656500,-79.377114,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Hotel,Electronics Store,Japanese Restaurant,Movie Theater,Fast Food Restaurant,Sandwich Place,Cosmetics Shop,Burger Joint
3,M5C,Downtown Toronto,43.669403,-79.372704,St. James Town,4,Coffee Shop,Pizza Place,Bakery,Grocery Store,Café,Bistro,Library,Beer Store,Filipino Restaurant,Playground
4,M5E,Downtown Toronto,43.647984,-79.375396,Berczy Park,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Breakfast Spot,Bakery


In [399]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['latitude'], dt_merged['longitude'], dt_merged['Neighborhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [400]:
dt_merged.loc[dt_merged['Cluster Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,0,Café,Park,Bookstore,Japanese Restaurant,Yoga Studio,Bank,Noodle House,Museum,Comfort Food Restaurant,College Gym


In [401]:
dt_merged.loc[dt_merged['Cluster Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Restaurant,Thai Restaurant,Pet Store,Pharmacy,Beer Store,Pool,Food Truck,Electronics Store,Sushi Restaurant
1,Downtown Toronto,1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,French Restaurant,Bank,Vegetarian / Vegan Restaurant,Bubble Tea Shop,Thai Restaurant,Japanese Restaurant
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Hotel,Electronics Store,Japanese Restaurant,Movie Theater,Fast Food Restaurant,Sandwich Place,Cosmetics Shop,Burger Joint
4,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Hotel,Italian Restaurant,Breakfast Spot,Bakery
5,Downtown Toronto,1,Boutique,Coffee Shop,Italian Restaurant,Café,Restaurant,French Restaurant,Spa,Hotel,Sushi Restaurant,Clothing Store
7,Downtown Toronto,1,Coffee Shop,Café,Gastropub,Clothing Store,Cosmetics Shop,Italian Restaurant,Lingerie Store,Hotel,Theater,Cocktail Bar
8,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Pizza Place,Gym,History Museum,Music Venue,Chinese Restaurant
9,Downtown Toronto,1,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Seafood Restaurant,Japanese Restaurant,Asian Restaurant,Bakery,Beer Bar
10,Downtown Toronto,1,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gym,American Restaurant,Japanese Restaurant,Seafood Restaurant,Deli / Bodega
12,Downtown Toronto,1,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Vietnamese Restaurant,Bakery,Taco Place,Nightclub,Pizza Place


In [402]:
dt_merged.loc[dt_merged['Cluster Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Park,Food Truck,Playground,Bike Trail,Women's Store,Doner Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant


In [403]:
dt_merged.loc[dt_merged['Cluster Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,3,Korean Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Ice Cream Shop,Café,Cocktail Bar,Karaoke Bar,Grocery Store,Dessert Shop


In [404]:
dt_merged.loc[dt_merged['Cluster Labels'] == 4, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,4,Coffee Shop,Pizza Place,Bakery,Grocery Store,Café,Bistro,Library,Beer Store,Filipino Restaurant,Playground
16,Downtown Toronto,4,Coffee Shop,Pizza Place,Bakery,Grocery Store,Café,Bistro,Library,Beer Store,Filipino Restaurant,Playground
